# Submit your review here 

https://forms.gle/75paKGVwGhu9y6Pi9

Remember, there are usually MANY ways to achieve things programatically. We will focus the reviews on substance and content. 

# Assignment 4: Merging

Related text: https://ledatascifi.github.io/ledatascifi-2021/content/03/05b_merging.html

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import seaborn as sns
# import statsmodels.api as sm

## PART 1

Insert cell(s) below this one as needed to finish this Part.

Load the following two datasets and answer these questions. Assume that the French data is the "left" dataset and the stock data is the "right" dataset. 

1. How many observations are there in `ff` data?
1. How many observations are there in `crsp` data?
4. After an inner merge?
1. How many observations are there after a left merge?
2. After a right merge? 
3. After an outer merge? 
1. Why isn't the answer to Q4 and Q5 the same?
1. Is this a 1:1, 1:M, M:1, or M:M merge?

Remember: Specify `how`, `on`, `indicator=True`, and `validate` on each merge!

In [2]:
ff = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily',start=1980,end=2010)[0] # the [0] is because the imported obect is a dictionary, and key=0 is the dataframe
ff = ff.reset_index().rename(columns={"Mkt-RF":"mkt_excess", "Date":"date"})
crsp = pd.read_stata('https://github.com/LeDataSciFi/ledatascifi-2021/blob/main/data/3firm_ret_1990_2020.dta?raw=true')
crsp['ret'] = crsp['ret']*100 # convert to precentage to match FF's convention on scaling


### My answers

I use the `merge_type` function from https://ledatascifi.github.io/ledatascifi-2021/content/03/05b_merging.html

And simply to avoid the copy-pasting the merge function over and over again, I use a lambda function. You can easily solve this without a lambda function, but it made my code simpler. 

In [3]:
quick_merge = lambda how: pd.merge(left=ff,right=crsp,on='date',how=how,indicator=True,validate="one_to_many")
#quick_merge('inner') is equiv to pd.merge(left=ff,right=crsp,on='date',how='inner',indicator=True,validate="one_to_many")

def merge_type(df1,df2,on):
    # if there are duplicates, dropping them will shrink the key vector
    if len(df1[on]) > len(df1[on].drop_duplicates()):
        _l = "many"
    else:
        _l = "one"
    if len(df2[on]) > len(df2[on].drop_duplicates()):
        _r = "many"
    else:
        _r = "one"
    return "{}_to_{}".format(_l,_r)

########################
# answers
########################

print(
f'''
Q1: N in L (ff):           {len(ff)}
Q2: N in R (crsp):         {len(crsp)}
Q3: N after Inner merge:   {len(quick_merge("inner"))}
Q4: N after L merge:       {len(quick_merge("left" ))}
Q5: N after R merge:       {len(quick_merge("right"))}
Q6: N after Outer merge:   {len(quick_merge("outer"))}
Q7: Q5 != Q4  b/c because the set of keys in L and R differ.
Q8: This merge is a:       {merge_type(ff,crsp,'date')}
'''
)


Q1: N in L (ff):           7571
Q2: N in R (crsp):         30236
Q3: N after Inner merge:   20172
Q4: N after L merge:       22700
Q5: N after R merge:       30236
Q6: N after Outer merge:   32764
Q7: Q5 != Q4  b/c because the set of keys in L and R differ.
Q8: This merge is a:       one_to_many



## PART 2: Creating variables around a merge 

Q9: Suppose you're analyzing the stock returns in the CRSP dataset. Add the FF variables to it. Then add a variable dataset that equals "the variance of the market return for the year" for each firm year. (The sentences are enough to pick which "how" option to choose.)  `describe()` the crsp dataset after you add the new variables - but you only need to describe `'ret','mkt_excess', 'SMB', 'HML'` and the new variance variable.

Note: You don't need to "annualize" the variance because the resulting data is daily.

HINT: If you get a mean of 1.334374 for the variance variable, that is wrong.

### My answer: The main point of this question is the following rule

Annual volatility of daily market returns must be calculated from a dataset with 1 observation per day. If you try to calculate volatility after merging it to the CRSP data, every day will be repeated 3 times (once per firm) and this will alter your calculations. 

**So, the general rule: Create variables on the "lowest level" datasets possible, and then merge up.**

In [4]:
# get the year (as a variable) in order compute the var for a year
ff['year'] = ff['date'].dt.year

# just so we can verify if we are right, let's figure out the within-year var
# compute variance within each year. 30 years of it:
display (ff.groupby('year')                      # for each year
        ['mkt_excess'].var()                     # compute variance
        [-10:].to_frame().style.format('{:.3f}') # unnecssary: just to print nicer
        )


,mkt_excess
year,
2000,2.417
2001,1.976
2002,2.527
2003,1.102
2004,0.518
2005,0.440
2006,0.449
2007,0.985
2008,6.344


Now I KNOW that all 2009 returns should be matched to a variance of 2.959! 

In [5]:
# save annual vol, then do the FF-CRSP merge, then merge in annual vol
ann_vol = ff.groupby('year')['mkt_excess'].var().reset_index()
ann_vol.columns = ['year','varExcess']

stock_analysis_df = pd.merge(crsp, ff, on='date',
                             how='left',validate='many_to_one')   # merge FF-CRSP
stock_analysis_df = pd.merge(stock_analysis_df, ann_vol, on='year',
                             how='left',validate='many_to_one')   # add annual vol
display(stock_analysis_df[['ret','mkt_excess', 'SMB', 'HML','varExcess']]
        .describe().style.format('{:,.3f}')) # prettier output, not necessary

# same output, but uses chains so no intermediate objects are named
# (crsp
#  .merge(ff,     on='date',how='left',validate='many_to_one')
#  .merge(ann_vol,on='year',how='left',validate='many_to_one')
#  [['ret','mkt_excess', 'SMB', 'HML','varExcess']].describe()  
#  .style.format('{:,.3f}')
# )

,ret,mkt_excess,SMB,HML,varExcess
count,"30,236.000","20,172.000","20,172.000","20,172.000","20,172.000"
mean,0.075,0.024,0.007,0.017,1.338
std,2.222,1.157,0.587,0.643,1.397
min,-51.869,-8.950,-4.180,-4.390,0.232
25%,-0.981,-0.480,-0.330,-0.260,0.440
50%,0.000,0.060,0.020,0.000,0.882
75%,1.074,0.550,0.340,0.270,1.546
max,33.228,11.350,4.490,4.830,6.344


In [6]:
# route 3 WRONG: calculate variance AFTER the merge
temp = pd.merge(left = crsp, right = ff,on='date',how='left', validate='many_to_one')
temp['year'] = temp['date'].dt.year
# the "transform" method does an operation on groups (after a groupby)
# but returns a dataset of the SAME size as before (rather than shrinking the dataset)
temp['wrongVar'] = temp.groupby('year')['mkt_excess'].transform(lambda x: x.var())
temp[['ret','mkt_excess', 'SMB', 'HML','wrongVar']].describe()

print('See! The 2009 values are wrong! Look:')
temp.query('year == 2009')['wrongVar'].head() 

See! The 2009 values are wrong! Look:


4791    2.950369
4792    2.950369
4793    2.950369
4794    2.950369
4795    2.950369
Name: wrongVar, dtype: float64

## PART 3: Adding a new variable via merge

Insert cell(s) below this one as needed to finish this Part.

Imagine you're analyzing our CCM dataset. (The code to download the CCM data is on 3.2.5 of the website.) Add a variable call "PatentStock" from [here](https://github.com/LeDataSciFi/ledatascifi-2021/blob/main/data/two_pat_vars.csv) to it. (The sentences are enough to pick which "how" option to choose in the merge.)

Q10: `.describe()` the "l_a" and the "PatentStock" variables after the merge. 
Q11: Tabulate the _merge variable. 

HINT: As you've seen several times before, pandas can download a csv file with just a URL. The "hitch" here is that I'm not giving you the URL. 

In [7]:
url = 'https://github.com/LeDataSciFi/ledatascifi-2021/blob/main/data/two_pat_vars.csv?raw=true'
patent = pd.read_csv(url)

In [8]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

url = 'https://github.com/LeDataSciFi/ledatascifi-2021/blob/main/data/CCM_cleaned_for_class.zip?raw=true'

#firms = pd.read_stata(url)   
# <-- that code would work, but GH said it was too big and
# forced me to zip it, so here is the work around to download it:

with urlopen(url) as request:
    data = BytesIO(request.read())

with ZipFile(data) as archive:
    with archive.open(archive.namelist()[0]) as stata:
        ccm = pd.read_stata(stata)

In [10]:
print(len(ccm))
ccm_with_pats = pd.merge(ccm,patent,how='left',
         on=['gvkey','fyear'],
         validate='one_to_one',
         indicator=True
        )
display(ccm_with_pats[['l_a','prodmktfluid']].describe())
print(ccm_with_pats['_merge'].value_counts())
print(len(ccm_with_pats))

# fluidity.columns # year, gvkey
# ccm.columns  # gvkey, fyear

223001


,l_a,prodmktfluid
count,222978.000000,88332.000000
mean,5.226964,7.479115
std,2.408634,3.909412
min,-6.907755,0.000000
25%,3.476993,4.626448
50%,5.094452,6.788517
75%,6.864064,9.634171
max,15.142903,40.775070


both          172315
left_only      50686
right_only         0
Name: _merge, dtype: int64
223001
